# Projeto VAD - [COMEÇO DE UMA VIDA]

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import pycountry
import dash
import dash_core_components as dcc
import dash_html_components as html
import re
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.subplots as sp
import time

from dash.exceptions import PreventUpdate


C:\Users\andre\AppData\Local\Temp\ipykernel_21284\4031837420.py:11: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

C:\Users\andre\AppData\Local\Temp\ipykernel_21284\4031837420.py:12: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



# Leitura de dados

In [2]:
df_antenatal = pd.read_csv('Data/antenatal.csv', encoding='ISO-8859-1', sep=';')
df_skilled = pd.read_csv('Data/Births attended by skilled health personnel.csv', encoding='ISO-8859-1', sep=';')
df_institutional = pd.read_csv('Data/Institutional births.csv', encoding='ISO-8859-1', sep=';')
df_married = pd.read_csv('Data/Women married or in a union before age 15 and 18.csv', encoding='ISO-8859-1', sep=';')
df_mortality = pd.read_csv('Data/GM-Child Mortality - Dataset - v11 - data-for-countries-etc-by-year.csv', encoding='ISO-8859-1', sep=',')
df_capita = pd.read_csv('Data/GM-GDP per capita - Dataset - v28 - data-for-countries-etc-by-year.csv',encoding='ISO-8859-1', sep=',')
df_history = pd.read_csv('Data/history.csv',encoding='ISO-8859-1', sep=';')

In [3]:
df_institutional.rename(columns = {'Percent':'Percent Institutional births'}, inplace = True)

In [4]:
df_mortality = df_mortality.query('Year >= 1985 and Year <= 2019')
df_capita = df_capita.query('Year >= 1985 and Year <= 2019')

### Merge de dados

In [5]:
df_merge = pd.merge(df_institutional, df_skilled, on=['Country', 'Year'], how='outer')
df_merge = pd.merge(df_merge, df_antenatal, on=['Country', 'Year'], how='outer')
df_merge = pd.merge(df_merge, df_mortality, on=['Country', 'Year'], how='outer')
df_merge = pd.merge(df_merge, df_capita, on=['Country', 'Year'], how='outer')
df_merge = df_merge.sort_values(by=['Country', 'Year'])



In [6]:
df_history.head()

Year                                           Moment_1 Moment_2  Moment_3  \
0  1985  Pregnancy Mortality Surveillance System (PMSS)...      NaN       NaN   
1  1986  Cancer and Steroid Hormone (CASH) Study report...      NaN       NaN   
2  1987  Pregnancy Risk Assessment Monitoring System (P...      NaN       NaN   
3  1988  Maternal and Child Health Epidemiology (MCHEPI...      NaN       NaN   
4  1989                                                NaN      NaN       NaN   

                                            Source_1 Source_2  Source_3  
0  https://www.cdc.gov/reproductivehealth/drh/pdf...      NaN       NaN  
1  https://www.cdc.gov/reproductivehealth/drh/pdf...      NaN       NaN  
2  https://www.cdc.gov/reproductivehealth/drh/pdf...      NaN       NaN  
3  https://www.cdc.gov/reproductivehealth/drh/pdf...      NaN       NaN  
4                                                NaN      NaN       NaN

In [7]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7515 entries, 4602 to 4497
Data columns (total 11 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Country                                                       7515 non-null   object 
 1   Year                                                          7515 non-null   float64
 2   Percent Institutional births                                  3295 non-null   float64
 3   Percent : Births attended by skilled health personnel (%)     2199 non-null   float64
 4   Percent : Antenatal care coverage - at least four visits (%)  2491 non-null   float64
 5   geo_x                                                         6720 non-null   object 
 6   Child mortality                                               6720 non-null   float64
 7   geo_y                                                         6650

In [8]:
df_merge[df_merge['Country'] == 'Portugal']

Country    Year  Percent Institutional births  \
6390  Portugal  1985.0                           NaN   
6391  Portugal  1986.0                           NaN   
6392  Portugal  1987.0                           NaN   
6393  Portugal  1988.0                           NaN   
6394  Portugal  1989.0                           NaN   
6395  Portugal  1990.0                           NaN   
6396  Portugal  1991.0                           NaN   
6397  Portugal  1992.0                           NaN   
6398  Portugal  1993.0                           NaN   
6399  Portugal  1994.0                           NaN   
6400  Portugal  1995.0                           NaN   
6401  Portugal  1996.0                           NaN   
6402  Portugal  1997.0                           NaN   
6403  Portugal  1998.0                           NaN   
6404  Portugal  1999.0                           NaN   
6405  Portugal  2000.0                           NaN   
6406  Portugal  2001.0                           NaN   
6407  Portugal  2002.0                           NaN   
6408  Portugal  2003.0                           NaN   
6409  Portugal  2004.0                           NaN   
6410  Portugal  2005.0                           NaN   
6411  Portugal  2006.0                           NaN   
6412  Portugal  2007.0                           NaN   
6413  Portugal  2008.0                           NaN   
6414  Portugal  2009.0                           NaN   
6415  Portugal  2010.0                           NaN   
4192  Portugal  2011.0                           NaN   
4193  Portugal  2012.0                           NaN   
2791  Portugal  2013.0                          99.0   
2792  Portugal  2014.0                          98.9   
2793  Portugal  2015.0                          98.9   
2794  Portugal  2016.0                          98.8   
2795  Portugal  2017.0                          98.8   
2796  Portugal  2018.0                          98.7   
4194  Portugal  2019.0                           NaN   
4195  Portugal  2020.0                           NaN   

      Percent : Births attended by skilled health personnel (%)  \
6390                                                NaN           
6391                                                NaN           
6392                                                NaN           
6393                                                NaN           
6394                                                NaN           
6395                                                NaN           
6396                                                NaN           
6397                                                NaN           
6398                                                NaN           
6399                                                NaN           
6400                                                NaN           
6401                                                NaN           
6402                                                NaN           
6403                                                NaN           
6404                                                NaN           
6405                                                NaN           
6406                                                NaN           
6407                                                NaN           
6408                                                NaN           
6409                                                NaN           
6410                                                NaN           
6411                                                NaN           
6412                                                NaN           
6413                                                NaN           
6414                                                NaN           
6415                                                NaN           
4192                                               99.9           
4193                                               99.9   

In [9]:
df_history.head()

Year                                           Moment_1 Moment_2  Moment_3  \
0  1985  Pregnancy Mortality Surveillance System (PMSS)...      NaN       NaN   
1  1986  Cancer and Steroid Hormone (CASH) Study report...      NaN       NaN   
2  1987  Pregnancy Risk Assessment Monitoring System (P...      NaN       NaN   
3  1988  Maternal and Child Health Epidemiology (MCHEPI...      NaN       NaN   
4  1989                                                NaN      NaN       NaN   

                                            Source_1 Source_2  Source_3  
0  https://www.cdc.gov/reproductivehealth/drh/pdf...      NaN       NaN  
1  https://www.cdc.gov/reproductivehealth/drh/pdf...      NaN       NaN  
2  https://www.cdc.gov/reproductivehealth/drh/pdf...      NaN       NaN  
3  https://www.cdc.gov/reproductivehealth/drh/pdf...      NaN       NaN  
4                                                NaN      NaN       NaN

In [10]:
events = []
for index, row in df_history.iterrows():
    year = row['Year']
    moment_1 = row['Moment_1']
    moment_2 = row['Moment_2']
    moment_3 = row['Moment_3']
    sources = [row['Source_1'], row['Source_2'], row['Source_3']]
    events.append({'year': year, 'moments': [moment_1, moment_2, moment_3], 'sources': sources})
    


# Dashboard

In [11]:


app = dash.Dash(__name__, suppress_callback_exceptions=True, external_stylesheets=[dbc.themes.BOOTSTRAP, 'style.css'])

year_options = [{'label': str(year), 'value': year} for year in sorted(df_merge['Year'].unique())]
col_options = [{'label': 'Antenatal care coverage - at least four visits (%)', 'value': 'Percent : Antenatal care coverage - at least four visits (%)'},
               {'label': 'Births attended by skilled health personnel (%)', 'value': 'Percent : Births attended by skilled health personnel (%)'},
               {'label': 'Percent Institutional births', 'value': 'Percent Institutional births'}]

app.layout = html.Div([
    dcc.Location(id='url', refresh=True),
    html.Div(id='page-content'),
    
                     
                     
])

index_page = html.Div([
    html.H1(children='The beginning of a life', style={
        'font-family': 'Calibri, sans-serif',
        'text-align': 'center',
        'color': '292b2c',
        'marginTop': '0.5cm'
    })
    ,

    html.Div(children=[
        dcc.Dropdown(id='col-dropdown',
                     options=col_options,
                     value='Percent : Antenatal care coverage - at least four visits (%)',
                     style={ 'maxWidth': '55%','margin': 'auto'}),

        dcc.Dropdown(id='year-dropdown',
                            options=[{'label': str(int(year)), 'value': year} for year in sorted(df_merge['Year'].unique())],
                            value=min(df_merge['Year']),
                            clearable=False,
                            style={ 'maxWidth': '22%','margin': 'auto', 'marginTop': '0.3cm'}),
        dcc.Graph(id='choropleth-map', style={'marginTop': '1cm'}),

        

        html.Div(className='row',style={'marginTop': '2cm'}, children=[
            html.Div(className='col', children=[
                dcc.Graph(id='bar-chart')
            ]),
            html.Div(className='col',style={'marginTop': '3cm'}, children=[
                html.Div(id='event-list')
            ])
        ]),
        html.Button('Voltar', id='back-button'),
    ])

])

country_page = html.Div([
    dcc.Dropdown(id='col-dropdown',
                     options=col_options,
                     value='Percent : Antenatal care coverage - at least four visits (%)',
                     style={'marginTop': '1cm'}),
    html.H1(id='country-header'),
    html.Div(id='marriage-info', style={'marginTop': '2cm'}),
    dcc.Graph(id='time-series-graph'),
    html.Div([
        html.H4("Animated Infant Mortality and population over decades"),
        html.P("Select an animation:"),
        dcc.RadioItems(
            id="selection",
            
            value="Infant Mortality - Scatter",
        ),
    ]),
    html.Button('Voltar', id='back-button'),
    
])

#marriage
@app.callback(Output('marriage-info', 'children'),
              [Input('url', 'pathname')])
def update_marriage_info(pathname):
    if pathname.startswith('/country/'):
        # Extrai o nome do país a partir do caminho da URL
        country_name = pathname.split('/')[-1]
    if country_name not in df_married['Country'].unique():
            return None
    row = df_married[df_married['Country'] == country_name].iloc[0]
    value = f"{row['Proportion of women aged 20-24 years who were married or in a union by age 15 (%)']}%"
    text = "Proportion of women aged 20-24 years who were married or in a union by age 15 in 2017:"
    if row['Proportion of women aged 20-24 years who were married or in a union by age 15 (%)'] > 0:
        return html.Div([
            html.H3(text, style={'font-family': 'Calibri', 'margin-top': '2cm', 'display': 'inline-block'}),
            html.Span(value, style={'color': 'red', 'font-size': '1.5em', 'display': 'inline-block', 'margin-left': '0em'}),
        ],style={'margin-left': '1em'}
        )
    else: 
        return None




@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname.startswith('/country/'):
        # Extrai o nome do país a partir do caminho da URL
        country_name = pathname.split('/')[-1]
        # Retorna a página do país selecionado
        return html.Div([
            
            html.H1(f'Country Details: {country_name}', id='country-header',style={
        'font-family': 'Calibri, sans-serif',
        'text-align': 'center',
        'color': '292b2c',
        'marginTop': '0.5cm'
    }),
            dcc.Dropdown(id='col-dropdown',
                     options=col_options,
                     value='Percent : Antenatal care coverage - at least four visits (%)',
                     style={ 'maxWidth': '55%','margin': 'auto'}),
            dcc.Graph(id='time-series-graph'),
            html.H1(id='marriage-info'),
            html.Div([
                
               html.H4('Infant Mortality vs GDP throughout the years', style={'text-align': 'center', 'margin-top': '1.5cm', 'margin': 'auto'}),
                dcc.RadioItems(
                    id="selection",
                    
                    value="Infant Mortality - Scatter",
                ),
                dcc.Loading(dcc.Graph(id="graph"), type="cube"),
            ], style = {'margin-top': '2cm'})
        ]),
    else:
        return index_page

#clicar    
@app.callback(Output('url', 'pathname'),
              [Input('choropleth-map', 'clickData'),
               Input('back-button', 'n_clicks')],
              [State('url', 'pathname')])
def on_url_change(clickData, n_clicks, current_path):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = None
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]


    if button_id == 'choropleth-map' and current_path != '/country' and clickData is not None: 
        country = clickData['points'][0]['location']
        return f'/country/{country}'
    elif button_id == 'back-button' and current_path == '/country':
        return '/'
    else:
        return current_path

#lista de eventos

@app.callback(
    Output('event-list', 'children'),
    [Input('year-dropdown', 'value')]
)
def update_event_list(year):
    df_year = df_history[df_history['Year'] == year]
    moments = [df_year['Moment_1'].values[0], df_year['Moment_2'].values[0], df_year['Moment_3'].values[0]]
    sources = [df_year['Source_1'].values[0], df_year['Source_2'].values[0], df_year['Source_3'].values[0]]
    events = [moment for moment in moments if not pd.isna(moment)]
    event_list = html.Ul([html.Li(f"{index + 1}. {event}") for index, event in enumerate(events)])
    source_list = html.Ul([
    html.Li([
        html.Span(f"Fonte_{i+1}: "),
        html.A(source, href=source, target='_blank')
    ]) for i, source in enumerate(sources) if source and not pd.isna(source)
])

    return html.Div([
        html.H3('Events with a potential effect on analysis: '),
        event_list if events else "No events recorded for this year.",
        html.H3('Sources:'),
        source_list if sources else "No source."
    ])

@app.callback(Output('country-header', 'children'),
              [Input('choropleth-map', 'clickData')])
def update_country_header(clickData):
    if clickData:
        country = clickData['points'][0]['location']
        return f'Detalhes do país: {country}'
    return ''


@app.callback(Output('time-series-graph', 'figure'),
              [Input('url', 'pathname'),
               Input('col-dropdown', 'value')])
def update_time_series_graph(pathname, col):
    if pathname.startswith('/country/'):
        country = pathname.split('/')[2]
        df_country = df_merge[df_merge['Country'] == country]
        x_min = df_country['Year'].min()
        x_max = df_country['Year'].max()

        return {
            'data': [go.Scatter(x=df_country['Year'], y=df_country[col], mode='lines+markers')],
            'layout': go.Layout( xaxis={'title': 'Year', 'range': [x_min-1, x_max +1]},
                                yaxis={'title': col, 'range': [0, 100]})
        }
    return go.Figure()


@app.callback(
    [Output('choropleth-map', 'figure'),
     Output('bar-chart', 'figure')],
    [Input('year-dropdown', 'value'),
     Input('col-dropdown', 'value')])
def update_charts(year, col):
    df_year = df_merge[df_merge['Year'] == year]

    choropleth_data = dict(
        type='choropleth',
        locations=df_year['Country'],
        locationmode='country names',
        z=df_year[col],
        colorscale='Blues',
        autocolorscale=False,
        marker_line_color='white',
        zmin=0,
        zmax=100,
        colorbar_title=col,

    )

    choropleth_layout = dict(
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='equirectangular'
        ),
        margin={'l': 0, 'r': 0, 't': 0, 'b': 0},
        height=400,
        coloraxis=dict(
            colorbar=dict(
                len=0.3,
                yanchor='top',
                y=-0.2,
                tickfont=dict(size=10),
                x=0.6,
                xpad=5
            ),
        ),
    )

    choropleth_fig = go.Figure(data=[choropleth_data], layout=choropleth_layout)
    
    top_5_countries = df_year.nsmallest(5, col)['Country']
    bar_data = [
        go.Bar(x=top_5_countries,
               y=df_year.loc[df_year['Country'].isin(top_5_countries), col],
               width=0.5)
    ]
    bar_layout = dict(
        title='The five countries with the worst results',
        xaxis=dict(title='Country'),
        yaxis=dict(title=col, range=[0, 100]),
        margin=dict(l=50, r=50, t=50, b=50)
    )
    bar_fig = go.Figure(data=bar_data, layout=bar_layout)

    return choropleth_fig, bar_fig


@app.callback(
    Output("graph", "figure"),
    [Input("selection", "value"), Input("url", "pathname")]
)
def display_animated_graph(selection, pathname):
    country = pathname.split("/")[-1] # extrai o nome do país do pathname
    df_country = df_merge[df_merge["Country"] == country]
    # converter a coluna Year em inteiros
    df_country["Year"] = df_country["Year"].astype(int)

    
    # encontra os valores mínimos e máximos para cada eixo
    x_min, x_max = df_country["Income per person"].min(), df_country["Income per person"].max()
    y_min, y_max = df_country["Child mortality"].min(), df_country["Child mortality"].max()

    animations = {
        "Infant Mortality - Scatter": px.scatter(
            df_country, # utiliza o dataframe filtrado pelo país
            x="Income per person",
            y="Child mortality",
            animation_frame="Year",
            animation_group="Country",

            color="Country",
            hover_name="Country",
            log_x=True,
            size=[1]*len(df_country),
            range_x=[x_min -100, x_max +100], # utiliza os valores mínimos e máximos de cada eixo
            range_y=[0, y_max + 100],
        ),
    }
    return animations[selection]



if __name__ == '__main__':
    app.run_server(debug=True, port=8050, use_reloader=False)




Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
